## 雪球大v速成神器：雪球组合同步代码
这是刚经过验证的最新的雪球组合同步代码，助力各位成为新一代雪球大v，源码参考了大神@food 的
[easytrader库](https://github.com/shidenggui/easytrader)

In [1]:
# -*- coding: utf-8 -*-
import requests
import json
import re

def snowCube(buyList,sellList):
#     自动获取存储在bmob后端云上的cookie    
#     s = requests.Session()
#     headers = {
#         'X-Bmob-Application-Id': '',
#         'X-Bmob-REST-API-Key': '',
#     }
#     p_url='https://api2.bmob.cn/1/classes/text/'
#     yourCookie = json.loads(s.get(p_url,headers=headers).text)['results'][1]['text']

    buyList=deal(buyList)
    sellList=deal(sellList)
    if len(buyList)==0 and len(sellList)==0:
        return
    
    pfls=[]
    
    pfl=dict()
    pfl['tag']=""
    pcook=yourCookie
    
    portfolio_code='ZH1332360' 
    pfl['cookies']=pcook
    pfl['cube']=portfolio_code
    pfls.append(pfl)

    for each in pfls: 
        snowTrade(buyList,sellList,each['cookies'],each['cube'],each['tag'])
    
def snowTrade(buyList,sellList,cookies,portfolio_code,tag):
    s = requests.Session()
    p_url='http://xueqiu.com/p/'+portfolio_code
    
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
        'Connection': 'keep-alive',
        'host': 'xueqiu.com',
        'Referer': 'https//http://xueqiu.com/',
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW 64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36 QIHU 360SE'
    }
    
    cookie_dict = {}
    for record in cookies.split(";"):
        key, value = record.strip().split("=", 1)
        cookie_dict[key] = value
        
    s.cookies = requests.utils.cookiejar_from_dict(cookie_dict)    
    resp = s.get(p_url,headers=headers).text
    portfolio_info = json.loads(re.search(r'(?<=SNB.cubeInfo = ).*(?=;\n)', resp).group())
    asset_balance = float(portfolio_info['net_value'])
    position = portfolio_info['view_rebalancing']
    cash = asset_balance * float(position['cash']) / 100 #可用资金
    market = asset_balance - cash
    p_info = [{
        'asset_balance': asset_balance,
        'current_balance': cash,
        'enable_balance': cash,
        'market_value': market,
        'money_type': u'CNY',
        'pre_interest': 0.25
    }]
    
    position_list = position['holdings']
    
    if len(sellList)>0:
        for position in position_list:
#                 print(position)
                if position['stock_symbol'] in sellList.keys():
                    print(position['stock_symbol'],sellList[position['stock_symbol']])
                    position['proactive'] = True
                    position['weight'] = 0#在这里自定义卖出仓位，范围0.01到1
                    position['volume'] = 0

                    
                    
    if len(buyList)>0:
        for k,v in buyList.items():
            stock = stockInfo(k,s,headers)
            position_list.append({
                    "code": stock['code'],
                    "name": stock['name'],
                    "enName": stock['enName'],
                    "hasexist": stock['hasexist'],
                    "flag": stock['flag'],
                    "type": stock['type'],
                    "current": stock['current'],
                    "chg": stock['chg'],
                    "percent": str(stock['percent']),
                    "stock_id": stock['stock_id'],
                    "ind_id": stock['ind_id'],
                    "ind_name": stock['ind_name'],
                    "ind_color": stock['ind_color'],
                    "textname": stock['name'],
                    "segment_name": stock['ind_name'],
                    "weight": round(cash*100/(len(buyList)+1),2),#在这里自定义买入仓位,范围0.01到1
                    "url": "/S/" + stock['code'],
                    "proactive": True,
                    "price": str(stock['current'])
                })
#     print(position_list)
    remain_weight = 100 - sum(i.get('weight') for i in position_list)
    cash = round(remain_weight, 2)
    data = {
        "cash": cash,
        "holdings": str(json.dumps(position_list)),
        "cube_symbol": str(portfolio_code),
        'segment': 'true',
        'comment': ""
    }
    try:
        resp = s.post("https://xueqiu.com/cubes/rebalancing/create.json",headers=headers,data=data)
    except Exception as e:
        print('调仓失败: %s ' % e)
        return
    else:
#         return
        print(resp.text)

    
def stockInfo(code,s,h):
        data = {
            'code': str(code),
            'size': '300',
            'key': '47bce5c74f',
            'market': 'cn',
        }
        r = s.get("https://xueqiu.com/stock/p/search.json",headers=h,params=data)
        stocks = json.loads(r.text)
        stocks = stocks['stocks']
        stock = None
        if len(stocks) > 0:
            stock = stocks[0]
        return stock


def deal(plist):
    rlist=dict()
    for k,v in plist.items():
        if k.startswith('300') or k.startswith('00'):
            rlist['SZ'+k[:6]]=v
        if k.startswith('6'):
            rlist['SH'+k[:6]]=v
    return rlist

SZ001979 sell
{"id":45417127,"status":"pending","cube_id":1366203,"prev_bebalancing_id":45304911,"category":"user_rebalancing","exe_strategy":"intraday_all","created_at":1537862461414,"updated_at":1537862461414,"cash_value":0.93617131,"cash":89.58,"error_code":"20842","error_message":"由于现在休市，组合处于「待成交」状态，请耐心等待。","error_status":"待成交","holdings":[{"stock_id":1001137,"weight":10.42,"segment_name":"通信","segment_id":7636001,"stock_name":"中兴通讯","stock_symbol":"SZ000063","segment_color":"#5db15d","proactive":false,"volume":0.00520817}],"rebalancing_histories":[],"comment":"@股美萌 调整了 1 只股票：\n卖出招商蛇口：41.25% → 0.00%","diff":0.0,"new_buy_count":0}


In [ ]:
#用法：在策略中先from xueqiu import * 然后就可以snowCube({买入dict}，{卖出dict})：
snowCube({},{'001979.XSHE':"sell"})
#缺少yourcookie的解决方法：自行填入自己登陆雪球后的cookie或微信搜索qev365关注公众号“万物量化”获得自动更新cookies神器